### Predict whether a voltage rise will occur or not in the next period . 

See [VoltageRiseBinTrain](voltageRiseBin_Train.ipynb) for the training of the RNN

---

#### Import modules to be used

In [1]:
#Import Modules 
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm # Profiling 
import seaborn as sbn
import pickle, sys, importlib,  time
import os
from pickle import load
import tensorflow as tf
import joblib

#### Import my own modules

In [2]:
# import psutil
function_folder = '../Modules/' 
# Add function folder to path if it is not already
if function_folder not in sys.path: sys.path.append(function_folder)

import oriFunctions as oriFc
import checker
import oriClass as oriCs
from oriVariables import (network_folder,
                          defAuth_hvBus_vRiseMax, 
                          defAuth_hvBus_vRiseMin, 
                          excel_folder, 
                          h_start_end,
                          train_split_date,  
                          testSet_end_date,
                          testSet_start_date, 
                          testSet_start_date_M2
                          )

#### Import Cleaned data files for component of the network

In [3]:
df_data = joblib.load(network_folder+'CleanedData.pkl')

# create daylight periods
per_index = df_data.index
per_daylight = ( pd.Series(index=per_index.to_timestamp(), dtype=object)
                .between_time(*h_start_end) ).index.to_period('10T')

# Extract daylight periods
df_data = df_data.loc[per_daylight]

# # Extract extract only testing set
df_final = df_data[(df_data.index>= testSet_start_date_M2) & (df_data.index<= testSet_end_date)]
per_index = df_final.index

# # Separate training and testing set 
df_test = df_final[per_index>=testSet_start_date_M2]

# Defined Voltage rise threshold
vm_mu_max, vm_mu_min = defAuth_hvBus_vRiseMax, defAuth_hvBus_vRiseMin  

#### Import the voltage rise from [VoltageRiseBinaryUpdated](VoltageRiseBinaryUpdated.ipynb)


In [4]:
numAndBin_vRise = joblib.load(network_folder+'simulationResults/Binary_Voltage_Rise.pkl')

var_bin_df = numAndBin_vRise.loc[per_index,['Volt_Rise_Bin']]

df_test_bin = pd.concat([df_test, var_bin_df], axis=1)

### Import RNN

In [6]:
# Load scaler and RNN from file
scaler = joblib.load(f'{network_folder}RNN/StLaurent_bin_vRise_scaler.plk')
lstm_model = tf.keras.models.load_model(f'{network_folder}RNN/StLaurent_bin_vRise_model')

#### Define history

In [7]:
df_test_bin.rename(columns={'Volt_Rise_Bin':'Volt_Rise_bin'}, inplace=True)# Rename column to avoid raising program warning
histTot = df_test_bin.rolling(lstm_model.input_shape[1])                   # Create a rolling windows to get the history

#### Predict values

In [8]:
hist_list = list(histTot)
len_hist = len(hist_list)
n0  = lstm_model.input_shape[1]                 # The first elem 
pred_per, pred = [],[]             
                
for hist_index in tqdm(range(n0, len_hist)):
    cur_hist = hist_list[hist_index]          # current hystory
    
    # run prediction for each period 
    pred_var, pred_per_var = oriFc.predictionBin_bloc(rnn_model=lstm_model, 
                                                      fitting_scaler=scaler, 
                                                      history= cur_hist, 
                                                      sig_thresh=0.5 )
    
    pred_per.append(pred_per_var)
    pred.append(pred_var)
    


100%|██████████| 26346/26346 [21:05<00:00, 20.81it/s]


#### Put predicted values in a pandas dataframe

In [9]:
predicted_values = pd.DataFrame(data = np.array(pred),
                                index=pred_per, 
                                columns = ['V_rise_Pred'] )

per_index3 = ( pd.Series(index=predicted_values.index.to_timestamp(), dtype=object
                           ).between_time('08:10','18:50') ).index.to_period('10T')

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

In [11]:
print(classification_report(var_bin_df.loc[per_index3],predicted_values.loc[per_index3]))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98     21241
           1       0.79      0.86      0.82      2549

    accuracy                           0.96     23790
   macro avg       0.88      0.92      0.90     23790
weighted avg       0.96      0.96      0.96     23790



In [12]:
print(confusion_matrix(var_bin_df.loc[per_index3],predicted_values.loc[per_index3]))

[[20647   594]
 [  356  2193]]


In [13]:
joblib.dump(predicted_values.loc[per_index3],f'{network_folder}simulationResults/Binary_Voltage_Rise_Predicted.pkl')

['../Pickle_files/simulationResults/Binary_Voltage_Rise_Predicted.pkl']